<a href="https://colab.research.google.com/github/dlfrnaos19/rock_scissors_paper_classifier/blob/main/%5BE_01%5DRockPaperScissor2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import glob
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt


In [ ]:
def resize_images(img_path):
    images = glob.glob(img_path + "/*.jpg")
    target_size = (28,28)
    for img in images:
        old = Image.open(img)
        new = old.resize(target_size, Image.ANTIALIAS)
        new.save(img,"jpeg")
    
    print(len(images), " Images resized.")


In [ ]:
resize_images(img_path+'scissors/')
resize_images(img_path+'rock/')
resize_images(img_path+'paper/')

In [ ]:
num_data = len(os.listdir('/content/rock/') + os.listdir('/content/scissors/') + os.listdir('/content/paper'))
def load_data(img_path, number_of_data):
    #가위 : 0, 바위 : 1, 보 : 2
    img_size = 28
    color = 3
    imgs = np.zeros(number_of_data * img_size * img_size * color, dtype=np.int32).reshape(number_of_data, img_size,img_size,color)
    labels = np.zeros(number_of_data, dtype=np.int32)

    idx = 0
    for file in glob.iglob(img_path+'scissors/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img # 데이터 영역에 이미지 복사
        labels[idx]=0 #가위 : 0
        idx = idx + 1
    
    for file in glob.iglob(img_path+'rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img # 데이터 영역에 이미지 복사
        labels[idx]=1 #바위 : 1
        idx = idx + 1
    
    for file in glob.iglob(img_path+'paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img # 데이터 영역에 이미지 복사
        labels[idx]=2 #보 : 2
        idx = idx + 1
    
    return imgs, labels

img_path = '/content/'
    


In [ ]:
images, labels = load_data(img_path, num_data)

In [ ]:
normed_images = images / 255

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(normed_images, labels,test_size=0.2,random_state=42 )

In [ ]:
# define sequence
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3,3),activation='relu', input_shape=(28,28,3))) # 16
model.add(tf.keras.layers.MaxPool2D(2,2))
model.add(tf.keras.layers.Conv2D(32,(3,3), activation='relu')) # 32
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32, activation='relu')) # 32
model.add(tf.keras.layers.Dense(3, activation='softmax'))


In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png')
plot_model(model, to_file='model_shapes.png', show_shapes=True)

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=15)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predicted_labels = np.argmax(predictions, axis=1)

In [ ]:
# get wrong prediction list
import random
wrong_predict_list = []
for i, _ in enumerate(predicted_labels):
    
    if predicted_labels[i] != y_test[i]:
        wrong_predict_list.append(i)

In [ ]:
# is prediction, label match
idx = 100
print(predictions[idx])
print(predicted_labels[idx])
print(y_test[idx])
plt.imshow(X_test[idx])

In [ ]:
samples = random.choices(population=wrong_predict_list, k=5)

In [ ]:
for n in samples:
    print("예측확률분포: ", str(predictions[n]))
    print("라벨 ", str(y_test[n])," 예측결과: ", str(predicted_labels[n]))
    plt.imshow(X_test[n], cmap=plt.cm.binary)
    plt.show()